In [1]:
%pip install -r "requirements_outlines.txt"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 8.5 MB/s  0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 11.1 MB/s  0:00:01m0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 5.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 17.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 3.8 MB/s  0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 4.8 MB/s  0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.9/887.9 MB 57.8 MB/s  0:00:12m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 62.1 MB/s  0:00:07m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 17.9 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 54.4 MB/s  0:00:016m0:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 5.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 70.6 MB/s 

In [1]:
import transformers
import accelerate
import outlines
import json
from outlines import from_transformers, Generator
from pydantic import BaseModel, Field, conlist
from typing import List, Literal, Annotated, Optional

/work/RuneEgeskovTrust#9638/miniconda3/envs/blame/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#model = from_transformers(
 #   transformers.AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.1-8B-Instruct", device_map="auto"),
  #  transformers.AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct"),
#)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
import torch, gc

# Delete the model object
del model
gc.collect()

# Clear PyTorch cache on GPU
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()


In [2]:
'''
This is the DeepSeek 14b model, which at first glance seems to perform better than the Llama model. 
Definitely worth considering if this should be used instead.
'''

model = from_transformers(
    transformers.AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-14B", device_map="auto"),
    transformers.AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-14B")
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:10<00:00,  2.50s/it]


In [3]:
# Defining the pydantic class which ensures the structured output from the llm
class BlameAnalysis(BaseModel):
    text: str = Field(description="The exact original sentence being analyzed")
    blame: bool = Field(description="Whether blame is present in the sentence")
    blamee: Optional[str] = Field(
        default=None,
        description="Who or what is being blamed (must not be empty if blame=true)"
    )
    arguments: Optional[str] = Field(
        default=None,
        description="What the blamee is being blamed for - the specific negative outcome (must not be empty if blame=true)"
    )

In [5]:
sentences = [
    # Blame
    "The government is responsible for the rising cost of healthcare.",
    "Corruption among politicians has led to the misuse of public funds.",
    "Corporate lobbying is to blame for weak environmental policies.",
    "The ruling party has failed to address income inequality.",
    "Ineffective leadership caused the crisis in public housing.",

    # No blame
    "Healthcare costs have been increasing steadily over the past decade.",
    "Income inequality continues to be a pressing social issue.",
    "Public housing availability has declined in recent years.",
    "Environmental policies remain a major topic of debate.",
    "The distribution of public funds is a central issue in current politics."
]


In [6]:
generator = Generator(model, BlameAnalysis)

In [8]:
for sentence in sentences:
    prompt = f"""Perform blame identification on the following sentence.
    Sentence: {sentence}

    Rules:
    - Start by determining whether blame is present at all in the sentence
    - Identify who is being blamed, what they are being blamed for, and the arguments used
    - Set blame=true ONLY if someone/something is being blamed for causing a negative outcome
    - The "text" field must be EXACTLY the sentence provided above - do not modify it
    - If blame=true, "blamee" must NOT be empty and "arguments" must contain the specific outcome they are blamed for
    - Do not leave arguments as an empty string

    Semantic roles:
    - Blamee: The patient receiving the blame (who or what is being blamed)
    - Argument: What is the blamee being blamed for (the negative outcome)

    Output your analysis in JSON format."""
    result = generator(prompt, max_new_tokens=128)
    data = json.loads(result)
    #print(json.dumps(data, indent=2))
    # Parsing json for saving
    result_out = BlameAnalysis.model_validate_json(result)
    # (Over)Writing to file to avoid duplicates
    with open("result_blame.json", "a") as f:
       json.dump(result_out.model_dump(), f, indent=2)


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [ ]:
data = json.loads(result)
print(json.dumps(data, indent=2))

In [ ]:
# Define the structured output schema for semantic role labeling
class Argument(BaseModel):
    role: str = Field(description="Semantic role (e.g., ARG0/Agent, ARG1/Patient, ARGM-TMP/Time, ARGM-LOC/Location)")
    text: str = Field(description="The text span filling this role")

class Predicate(BaseModel):
    predicate: str = Field(description="The main predicate/verb")
    agent: str = Field(description="ARG0: The entity performing the action (who/what does it)")
    patient: str = Field(description="ARG1: The entity affected by the action (who/what is affected)", default="")
    arguments: List[Argument] = Field(description="List of semantic arguments for this predicate")

class SemanticRoleLabeling(BaseModel):
    sentence: str = Field(description="The original sentence")
    predicates: List[Predicate] = Field(description="List of predicates with their semantic roles")

# Create a structured generator
generator = Generator(model, SemanticRoleLabeling)

# Example text for semantic role labeling
text = "John ate an apple in the park yesterday."

# Create the prompt
prompt = f"""Perform semantic role labeling on the following sentence. Identify all predicates and their semantic arguments.

Sentence: {text}

Common semantic roles:
- ARG0: Agent (who performs the action)
- ARG1: Patient/Theme (what is affected by the action)
- ARG2: Recipient, Beneficiary, or Instrument
- ARGM-TMP: Temporal (when)
- ARGM-LOC: Location (where)
- ARGM-MNR: Manner (how)
- ARGM-PRP: Purpose (why)

Analyze the sentence and output the semantic role labeling in the specified JSON format."""

# Generate structured output
result = generator(prompt, max_new_tokens=2048)

'''
# Print the results
print("Original sentence:", result.sentence)
print("\nSemantic Role Labeling:")
for pred in result.predicates:
    print(f"\nPredicate: {pred.predicate}")
    for arg in pred.arguments:
        print(f"  {arg.role}: {arg.text}")

'''

In [2]:
import pandas as pd
df = pd.read_csv("annotation_data.csv")

In [4]:
df.head()

,version https://git-lfs.github.com/spec/v1
0,oid sha256:3fccfc711d5867025434f4f07129234c7ea...
1,size 68627460
